In [1]:
# --- Step 1: Environment Setup & Data Ingestion for SINADEF ---
import pandas as pd
import numpy as np

print("Libraries imported successfully.")

# Define the path to the raw SINADEF Excel file
sinadef_excel_path = '../data/fallecidos_sinadef.xlsx'

try:
    # Load the Excel file. SINADEF datasets can be large, so this might take a moment.
    df_sinadef_raw = pd.read_excel(sinadef_excel_path, engine='openpyxl')
    print(f"Successfully loaded raw data from '{sinadef_excel_path}'.")
    print(f"Initial shape: {df_sinadef_raw.shape[0]} rows, {df_sinadef_raw.shape[1]} columns.")
    
    print("\n--- Initial Data Preview (First 5 Rows) ---")
    display(df_sinadef_raw.head())
    
    print("\n--- General DataFrame Info ---")
    df_sinadef_raw.info(verbose=True, show_counts=True)

except FileNotFoundError:
    print(f"CRITICAL ERROR: The file was not found at the specified path: {sinadef_excel_path}")

Libraries imported successfully.
Successfully loaded raw data from '../data/fallecidos_sinadef.xlsx'.
Initial shape: 1048059 rows, 32 columns.

--- Initial Data Preview (First 5 Rows) ---


,NÂº,TIPO SEGURO,SEXO,EDAD,TIEMPO EDAD,ESTADO CIVIL,NIVEL DE INSTRUCCIÃ“N,ETNIA,COD# UBIGEO DOMICILIO,PAIS DOMICILIO,...,DEBIDO A (CAUSA B),CAUSA B (CIE-X),DEBIDO A (CAUSA C),CAUSA C (CIE-X),DEBIDO A (CAUSA D),CAUSA D (CIE-X),DEBIDO A (CAUSA E),CAUSA E (CIE-X),DEBIDO A (CAUSA F),CAUSA F (CIE-X)
0,1,ESSALUD,MASCULINO,54,AÃ‘OS,CASADO,SECUNDARIA COMPLETA,MESTIZO,92-33-14-01-13-000,PERU,...,FASCEITIS NECROTIZANTE,SIN REGISTRO,ESCARA SACRA,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,DIABETES MELLITUS,SIN REGISTRO,INSUFICIENCIA RENAL,SIN REGISTRO
1,2,SIS,MASCULINO,41,AÃ‘OS,SOLTERO,PRIMARIA COMPLETA,MESTIZO,92-33-04-01-14-000,PERU,...,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
2,3,ESSALUD,MASCULINO,84,AÃ‘OS,CASADO,SECUNDARIA COMPLETA,MESTIZO,92-33-14-01-01-000,PERU,...,INSUFICIENCIA RESPITATORIA,J960,SEPSIS,A418,NEUMONIA,J188,CAQUEXIA,R64X,SIN REGISTRO,SIN REGISTRO
3,4,IGNORADO,FEMENINO,82,AÃ‘OS,SOLTERO,PRIMARIA INCOMPLETA,SIN CLASIFICACIÃ“N,92-33-14-01-32-000,PERU,...,FALLA MULTIORGANICA,R99X,SIN REGISTRO,J189,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO
4,5,IGNORADO,MASCULINO,54,AÃ‘OS,SOLTERO,SUPERIOR NO UNIV. COMP.,SIN CLASIFICACIÃ“N,92-33-14-01-32-000,PERU,...,ESTRECHEZ CONGENITA DEL CONDUCTO CERVICAL,M480,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO,SIN REGISTRO



--- General DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048059 entries, 0 to 1048058
Data columns (total 32 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   NÂº                     1024307 non-null  object
 1   TIPO SEGURO             1047388 non-null  object
 2   SEXO                    1046028 non-null  object
 3   EDAD                    1037476 non-null  object
 4   TIEMPO EDAD             1036587 non-null  object
 5   ESTADO CIVIL            1035781 non-null  object
 6   NIVEL DE INSTRUCCIÃ“N   1034336 non-null  object
 7   ETNIA                   1034089 non-null  object
 8   COD# UBIGEO DOMICILIO   1029754 non-null  object
 9   PAIS DOMICILIO          1029485 non-null  object
 10  DEPARTAMENTO DOMICILIO  1025963 non-null  object
 11  PROVINCIA DOMICILIO     1025948 non-null  object
 12  DISTRITO DOMICILIO      1022351 non-null  object
 13  FECHA                   1022500 non-null

In [6]:
# --- VERSIÓN FINAL Y ROBUSTA de la función de SINADEF ---
def process_sinadef_data(raw_df):
    df = raw_df.copy()
    print("SINADEF processing pipeline initiated...")
    
    # 1. Estandarizar
    df.columns = (df.columns.str.lower().str.replace(' ', '_', regex=False).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
    
    # 2. Seleccionar y Renombrar (AHORA USAMOS DOMICILIO)
    columnas_de_interes = {
        'departamento_domicilio': 'departamento',
        'provincia_domicilio': 'provincia',
        'distrito_domicilio': 'distrito',
        'edad': 'edad',
        'causa_a_cie-x': 'causa_cie10' # Ajusta este nombre si la celda detective te da otro
    }
    columnas_a_usar = {k: v for k, v in columnas_de_interes.items() if k in df.columns}
    df = df[list(columnas_a_usar.keys())].rename(columns=columnas_a_usar)

    # 3. Limpiar Nulos CRÍTICOS PRIMERO
    # Eliminamos las filas donde la ubicación del domicilio es desconocida, ya que son inútiles para la agregación.
    df.dropna(subset=['departamento', 'provincia', 'distrito'], inplace=True)
    
    # 4. Limpieza de texto y edad
    df['departamento'] = df['departamento'].str.strip()
    df['provincia'] = df['provincia'].str.strip()
    df['distrito'] = df['distrito'].str.strip()
    df['edad'] = pd.to_numeric(df['edad'], errors='coerce')
    df.dropna(subset=['edad'], inplace=True)
    df['edad'] = df['edad'].astype(int)

    # 5. Crear UBIGEO
    df['ubigeo'] = df['departamento'] + '_' + df['provincia'] + '_' + df['distrito']
    
    # 6. Feature Engineering
    lista_codigos_prevenibles = ['A0', 'A1', 'A3', 'B15', 'B16', 'B17', 'B20', 'E4']
    if 'causa_cie10' in df.columns:
        df['es_causa_prevenible'] = df['causa_cie10'].astype(str).str.startswith(tuple(lista_codigos_prevenibles)).astype(int)
    else:
        df['es_causa_prevenible'] = 0

    # 7. Agregación
    df_aggregated = df.groupby('ubigeo').agg(
        s_factor_total_muertes=('edad', 'count'),
        s_factor_edad_prom_muerte=('edad', 'mean'),
        s_factor_tasa_prevenibles=('es_causa_prevenible', 'mean')
    ).reset_index()

    print("\nSINADEF processing and aggregation complete!")
    return df_aggregated

In [7]:
# --- Step 3: Execute Pipeline and Export Aggregated Asset ---

# Verificamos que el DataFrame crudo exista antes de proceder
if 'df_sinadef_raw' in locals():
    # Llamamos a la función que hemos construido y ajustado
    sinadef_aggregated_df = process_sinadef_data(df_sinadef_raw)
    
    # Guardamos el resultado final y agregado en un nuevo archivo CSV
    aggregated_path = '../data/sinadef_aggregated_v1.csv'
    sinadef_aggregated_df.to_csv(aggregated_path, index=False)
    
    print("\n----------------------------------------------------")
    print("✅  PROCESO DE SINADEF FINALIZADO CON ÉXITO")
    print(f"Archivo '{aggregated_path}' ha sido exportado.")
    print("Este archivo contiene los S-Factors agregados y está listo para la fusión.")
    
    print("\n--- Muestra del Dataset Agregado Final ---")
    display(sinadef_aggregated_df.sample(10, random_state=42)) # Mostramos una muestra aleatoria
    
    print("\n--- Estadísticas Descriptivas de los S-Factors ---")
    display(sinadef_aggregated_df.describe().T)

else:
    print("No se puede proceder. El DataFrame crudo de SINADEF (`df_sinadef_raw`) no fue cargado correctamente.")

SINADEF processing pipeline initiated...

SINADEF processing and aggregation complete!

----------------------------------------------------
✅  PROCESO DE SINADEF FINALIZADO CON ÉXITO
Archivo '../data/sinadef_aggregated_v1.csv' ha sido exportado.
Este archivo contiene los S-Factors agregados y está listo para la fusión.

--- Muestra del Dataset Agregado Final ---


,ubigeo,s_factor_total_muertes,s_factor_edad_prom_muerte,s_factor_tasa_prevenibles
188,ANCASH_HUARMEY_HUAYAN,45,75.733333,0.0
1320,JUNIN_JAUJA_SINCOS,176,65.090909,0.0
2221,TUMBES_TUMBES_PAMPAS DE HOSPITAL,245,70.665306,0.0
1087,HUANUCO_DOS DE MAYO_MARIAS,111,58.027027,0.0
674,CAJAMARCA_CELENDIN_SUCRE,108,63.212963,0.0
535,AYACUCHO_HUANTA_PUCACOLPA,28,58.642857,0.0
906,CUSCO_PARURO_COLCHA,68,78.602941,0.0
1441,LA LIBERTAD_SIN REGISTRO_SIN REGISTRO,7,60.428571,0.0
1114,HUANUCO_HUAMALIES_SINGA,111,61.009009,0.0
1875,PIURA_HUANCABAMBA_LALAQUIZ,145,69.282759,0.0



--- Estadísticas Descriptivas de los S-Factors ---


,count,mean,std,min,25%,50%,75%,max
s_factor_total_muertes,2268.0,450.652116,1669.614429,1.0,26.750000,81.000000,222.000000,29515.0
s_factor_edad_prom_muerte,2268.0,65.731287,42.345484,2.0,61.471414,66.851953,70.844697,2021.0
s_factor_tasa_prevenibles,2268.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
